In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd

In [2]:
# Save Executable Path
executable_path = {"executable_path": "chromedriver"}

# Save Browser Instance
browser = Browser("chrome", **executable_path, headless=False)

# ff_df URL
url = "https://fantasy.espn.com/football/league/standings?leagueId=63563391&seasonId=2020"

# Visit URL
browser.visit(url)

In [3]:
# Save As HTML
html = browser.html

# Parse HTML
soup = bs(html, "html.parser")

# Find All Team Names
teams = soup.find_all('span', class_='teamName')[0:12]

# Empty List to Hold Team Names
team_names = []

# Iterate Through Found Team Names
for x in range(len(teams)):
    
    # Append Each Name to the Empty List
    team_names.append(teams[x].text)

In [4]:
# Function to Get Columns by Class Name
def getColumns(COLUMN):
    
    # Empty List to Hold Values
    inner_list = []
    
    # Save All Found Values
    values = soup.find_all('div', class_=COLUMN)[-12:]

    # Iterate Through Found Values
    for x in range(len(values)):
        
        # Append Each Value to Empty List
        inner_list.append(values[x].text)

    # Return List
    return inner_list

# Get the points-for Column
points_for = getColumns('points-for')

# Get the points-against Column
points_against = getColumns('points-against')

# Get the moves Column
moves = getColumns('moves')

In [5]:
# Save All Records Found
records = soup.find_all('div', class_='recordSplit')[3:]

# Empty List to Hold Division Record
division_record = []

# Empty List to Hold Home Record
home_record = []

# Empty List to Hold Away Record
away_record = []

# Iterate Through Records
for x in range(len(records)):
    
    # Handle Division Records
    if x % 3 == 0:
        
        # Append Division Record to List
        division_record.append(records[x].text)
        
    # Handle Home Records
    elif x % 3 == 1:
        
        # Append Home Record to List
        home_record.append(records[x].text)
        
    # Handle Away Records
    elif x % 3 == 2:

        # Append Away Record to List
        away_record.append(records[x].text)

In [6]:
# Create DataFrame From Scraped Values
ff_df = pd.DataFrame(
    {'team_names': team_names,
     'points_for': points_for,
     'points_against': points_against,
     'division_record': division_record,
     'home_record': home_record,
     'away_record': away_record,
     'moves': moves,
    })

# Save DataFrame to CSV
ff_df.to_csv(r"../Data/ff_df.csv", index = False)

In [7]:
# ff_df URL
url = "https://fantasy.espn.com/football/league/schedule?leagueId=63563391"

# Visit URL
browser.visit(url)

In [8]:
# Save As HTML
html = browser.html

# Parse HTML
soup = bs(html, "html.parser")

# Empty List to Hold Scores
score = []

# Find All Scores
scores = soup.find_all('span', class_='link')

# Iterate Through Found Scores
for x in range(len(scores)):
    # Handle Bye
    if x == 166:
        
        # Append Bye
        score.append("Bye")
        
    # Handle Bye
    elif x == 167:
        
        # Append Bye
        score.append("Bye")
        
        # Append Text
        score.append(scores[x-1].text)
        
        # Append Text
        score.append(scores[x].text)
        
    # Handle Standard Cases
    else:
        
        # Append Text
        score.append(scores[x].text)
        
# Empty List to Hold Names
name = []

# Save All Found Names
names = soup.find_all('span', class_='teamName')


# Iterate Through Found Names
for x in range(len(names)):
    
    # Append Names to Empty List
    name.append(names[x].text)

# Create DataFrame
scores_df = pd.DataFrame({"team_name":name, "scores": score})

# Empty List to Hold Weeks
weeks = []

# Iterate Through Season Length
for x in range(16):
    
    # Iterate For Each Week
    for n in range(12):
        
        # Append Week Name
        weeks.append(f"Week {x+1}")
        
# Save Created List to DataFrame
scores_df["weeks"] = weeks

# Empty List to Hold Opponent Name
opponent = []

# Empty List to Hold Opponent Score
opponent_score = []

# Iterate For Every Row
for x in range(len(scores_df)):
    
    # Handle Web Format
    if x % 2 == 0:
        
        # Append Opponent Name
        opponent.append(scores_df["team_name"][x+1])
        
        # Append Opponent Score
        opponent_score.append(scores_df["scores"][x+1])
        
    # Handle Web Format
    elif x % 2 == 1:
        
        # Append Opponent Name
        opponent.append(scores_df["team_name"][x-1])
        
        # Append Opponent Score
        opponent_score.append(scores_df["scores"][x-1])
        
# Save Opponent Names to DataFrame
scores_df["opponent"] = opponent

# Save Opponent Scores to DataFrame
scores_df["opponent_score"] = opponent_score


# Rename Columns
scores_df.columns = ['team_name', 'score', 'week', 'opponent', 'opponent_score']

# Save to CSV 
scores_df.to_csv(r"../Data/scores_df.csv", index = False)

In [9]:
# divisions URL
url = "https://fantasy.espn.com/football/league/standings?leagueId=63563391"

# Visit URL
browser.visit(url)

In [10]:
# Save As HTML
html = browser.html

# Parse HTML
soup = bs(html, "html.parser")

# Empty List to Hold Divisions
team_divisions = []

# Find All Divisions
divs = soup.find_all('div', class_='Table__Title')

# Iterate Through All Found Divisions
for x in range(len(divs)):
    
    # Append Division to List
    team_divisions.append(divs[x].text)
    
# Empty List to Hold Team Names
team_names = []

# Find All Team Names
names = soup.find_all('span', class_='teamName')

# Iterate Through All Found Team Names
for x in range(len(names)):
    
    # Append Team Name to List
    team_names.append(names[x].text)
    
# Empty List to Hold Divisions
divisions = []

# Iterate Through All Found Team Names
for x in range(len(team_names) +1):
    
    # Handle First Division
    if x < 6:
        
        #Append Division Name
        divisions.append(team_divisions[0])
        
    # Handle Second Division
    elif x > 6:
        
        #Append Division Name
        divisions.append(team_divisions[1])

In [11]:
# Save to DataFrame
divisions = pd.DataFrame(divisions, columns = ["Division"], index = team_names)

# Reset Index
divisions.reset_index(inplace=True)

# Rename Columns
divisions.columns = ["team_names", "Division"]

# Save to CSV
divisions.to_csv(r"../Data/divisions.csv", index = False)